In [1]:
# read from preprocessed customer and customer test
from fastai.learner import *
from fastai.column_data import *
from fastai.structured import *

D:\python\envs\fastai\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
PATH=Path('data/elo')
# C drive path
PATH1 = Path('c:/data/')

In [4]:
customer = pd.read_feather(f'{PATH1}\customer')
customer_test = pd.read_csv(f'{PATH1}\customer_test1.csv')

In [10]:
# customer.head()
customer['purchase_amount'].fillna(0.0,inplace=True)
customer_test['purchase_amount'].fillna(0.0,inplace=True)


In [38]:
customer_test.head()

,card_id,cust_index,merch_index,purchase_amount
0,C_ID_b3334831be,0,775,-2.846532
1,C_ID_af9b01c5b0,0,775,-2.846532
2,C_ID_22f6678098,0,775,-2.846532
3,C_ID_dcdf88cb4d,2,1896,-0.581616
4,C_ID_92b63a03a6,3,3499,-0.596643


In [28]:
del(customer_test['index'])

In [5]:
customer_test['cust_index'].max()

493.0

In [29]:
customer_test['cust_index'].fillna(0,inplace=True)

In [30]:
customer_test['merch_index'].fillna(0,inplace=True)

In [31]:
customer_test['cust_index'] = customer_test['cust_index'].astype(np.uint16)
customer_test['merch_index'] = customer_test['merch_index'].astype(np.uint16)

In [3]:
customer_test.to_csv(f'{PATH1}\customer_test1.csv')
customer.to_feather(f'{PATH1}\customer.csv')

NameError: name 'customer_test' is not defined

In [5]:
cat_vars = ['cust_index','merch_index']
contin_vars = ['purchase_amount']

In [8]:
customer.loc[customer['target'] < 0, 'target'] = 0


In [9]:
from sklearn import preprocessing

def Normalizing(df):
    x = df['purchase_amount'].values #returns a numpy array
    x =  x.reshape(-1, 1) 
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df['purchase_amount'] = pd.DataFrame(x_scaled)
    return df 

In [11]:
customer = Normalizing(customer)


In [12]:
customer_test = Normalizing(customer_test)

In [13]:
len(customer_test)

1854045

In [14]:
dep = 'target'
df_train = customer[cat_vars+contin_vars+[dep]].copy()

In [15]:
customer_test[dep] = 0.0
df_test = customer_test[cat_vars+contin_vars+[dep, 'card_id']].copy()

In [16]:
for v in cat_vars: df_train[v] = df_train[v].astype('category').cat.as_ordered()
apply_cats(df_test, df_train)

In [17]:
for v in contin_vars:
    df_train[v] = df_train[v].fillna(0).astype('float32')
    df_test[v] = df_test[v].fillna(0).astype('float32')

In [18]:
samp_size = len(df_train)
df, y, nas, mapper = proc_df(df_train, dep, do_scale=True)

In [19]:
df_test1, _, nas, mapper = proc_df(df_test, dep, do_scale=True,skip_flds=['card_id'],mapper=mapper, na_dict=nas)

In [20]:
train_ratio = 0.75
# train_ratio = 0.9
train_size = int(samp_size * train_ratio); train_size
val_idx = list(range(train_size, len(df)))

In [21]:
def exp_rmse(y_pred, targ):
#     targ = inv_y(targ)
    pct_var = (targ - y_pred)
    return math.sqrt((pct_var**2).mean())

y_range = (0, 1)

In [22]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, y.astype(np.float32), cat_flds=cat_vars, bs=2056,
                                       test_df=df_test1)

In [23]:
cat_sz = [(c, len(df_train[c].cat.categories)+1) for c in cat_vars]
cat_sz

[('cust_index', 496), ('merch_index', 38366)]

In [24]:
emb_szs = [(c, min(200, (c+1)//2)) for _,c in cat_sz]
emb_szs

[(496, 200), (38366, 200)]

In [25]:
m = md.get_learner(emb_szs,len(df.columns)-len(cat_vars),0.04,1,[1000,500],[0.001,0.01])

In [141]:
lr=1e-5
m.lr_find()
m.sched.plot(100)

  3%|█▋                                                               | 1151/43429 [00:16<10:00, 70.36it/s, loss=0.752]

KeyboardInterrupt: 

In [26]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-4

In [28]:
m.fit(lr, 3,metrics=[exp_rmse])

epoch      trn_loss   val_loss   exp_rmse                                                                              
    0      0.68444    0.794257   0.84241   
    1      0.656671   0.793095   0.839226                                                                              
    2      0.654127   0.796014   0.841141                                                                              



[array([0.79601]), 0.8411410420520833]

In [29]:
x,y=m.predict_with_targs()
exp_rmse(x,y)
pred_test=m.predict(True)
pred_test = np.exp(pred_test)
df_test[dep]=pred_test

In [99]:
 import lightgbm as lgb

In [100]:
((val, trn), (y_val, y_trn)) = split_by_idx(val_idx, df.values, y)
lgb_train = lgb.Dataset(trn, y_trn)
lgb_val = lgb.Dataset(val, y_val)   

In [101]:
    params = {
        #'objective': 'binary',
        'objective': 'regression',
        'boosting': 'gbdt',
        'learning_rate': 0.2,
        'verbose': 0,
        'num_leaves': 100,
        'bagging_fraction': 0.95,
        'bagging_freq': 1,
        'bagging_seed': 1,
        'feature_fraction': 0.9,
        'feature_fraction_seed': 1,
        'max_bin': 256,
        'num_rounds': 100,
        'metric': 'l2_root',
        'num_iterations': 100,
        'num_threads':0,
        'device_type':'cpu' #gpu
    }


In [102]:
lgbm_model = lgb.train(params, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=5)
predictions = lgbm_model.predict(df_test1)
df_test[dep] =predictions

D:\python\envs\fastai\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[5]	valid_0's rmse: 0.879855
[10]	valid_0's rmse: 0.880243
[15]	valid_0's rmse: 0.880571
[20]	valid_0's rmse: 0.880946
[25]	valid_0's rmse: 0.881693
[30]	valid_0's rmse: 0.881804
[35]	valid_0's rmse: 0.881972
[40]	valid_0's rmse: 0.882292
[45]	valid_0's rmse: 0.882711
[50]	valid_0's rmse: 0.882888
[55]	valid_0's rmse: 0.883226
[60]	valid_0's rmse: 0.883374
[65]	valid_0's rmse: 0.883422
[70]	valid_0's rmse: 0.883665
[75]	valid_0's rmse: 0.883689
[80]	valid_0's rmse: 0.884034
[85]	valid_0's rmse: 0.884071
[90]	valid_0's rmse: 0.884397
[95]	valid_0's rmse: 0.884517
[100]	valid_0's rmse: 0.884749


In [30]:
df_result = df_test.groupby(['card_id']).agg({'target' : np.mean}).reset_index().copy()

In [31]:
len(df_result)

123603

In [42]:
csv_fn=f'{PATH}/tmp/elo-subv7.csv'

In [110]:
df_result[['card_id', dep]].to_csv(csv_fn, index=False)



In [33]:
csv_fn5=f'{PATH}/tmp/elo-subv5.csv'

In [34]:
result_all = pd.read_csv(csv_fn5)

In [35]:
result_all =pd.merge(df_result,result_all,how='outer',on=['card_id'])

In [36]:
len(result_all)

123623

In [37]:
result_all.head()

,card_id,target_x,target_y
0,C_ID_0001238066,1.0,0.252285
1,C_ID_0001793786,1.0,0.377866
2,C_ID_00024e244b,1.0,0.692639
3,C_ID_0002709b5a,1.0,0.365717
4,C_ID_000298032a,1.0,0.494499


In [38]:
# result_all.loc[result_all['target_x'].isnull(),'target_x'] = 
# result_all['target_x'].apply(lambda row: row.a + row.b, axis=1)
result_all.target_x = np.where( result_all.target_x.isnull(),result_all.target_y, result_all.target_x)


In [39]:
del(result_all['target_y'])

In [40]:
result_all.rename(columns={'target_x':'target'}, inplace=True)

In [43]:
result_all[['card_id', dep]].to_csv(csv_fn, index=False)

In [ ]:
df_try_unique = pd.read_csv(csv_fn, low_memory=False)
df_try_unique = df_try_unique.groupby('fullVisitorId').mean()
df_try_unique = df_try_unique.reset_index()
df_uniq1[['card_id',dep]].to_csv(csv_fn, index=False)